### Notebook to demonstrate TAO Object Detection workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Sample prediction for an Object Detection model
<img align="center" src="../example_images/sample_object_detection.jpg" width="960">

### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Running dataset convert
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain
    - Export
    - Tao-Deploy
    - Inference on TAO
    - Inference on TRT
    
### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Dataset convert Action](#head-3)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [View hyperparameters that are enabled by default](#head-8)
1. [Set AutoML related configurations](#head-9)
1. [Actions](#head-10)
1. [Train](#head-11)
1. [Evaluate](#head-12)
1. [Optimize: Apply specs for prune](#head-14)       
1. [Optimize: Apply specs for retrain](#head-15)
1. [Optimize: Run actions](#head-16)
1. [Export](#head-17)
1. [TRT Engine generation using TAO-Deploy](#head-19)
1. [TAO inference](#head-20)
1. [TRT inference](#head-21)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a model_name in FIXME 1
2. Assign a workdir in FIXME 2
3. Assign the ip_address and port_number in FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
4. Assign the ngc_api_key variable in FIXME 4
5. (Optional) Enable AutoML if needed in FIXME 5
6. Choose between default and custom dataset in FIXME 6
7. Assign path of DATA_DIR in FIXME 7
8. Choose between Bayesian and Hyperband automl_algorithm in FIXME 8 (If automl was enabled in FIXME5)

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. deformable_detr - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/deformable_detr.html
# 2. detectnet_v2 - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html
# 3. dino - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/dino.html
# 4. dssd - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/ssd.html
# 5. efficientdet_tf1 - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/efficientdet_tf1.html
# 6. efficientdet_tf2 - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/efficientdet_tf2.html
# 7. faster_rcnn - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/fasterrcnn.html
# 8. retinanet - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/retinanet.html
# 9. ssd - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/ssd.html
# 10. yolo_v3 - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v3.html
# 11. yolo_v4 - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html
# 12. yolo_v4_tiny - https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4_tiny.html

model_name = "detectnet_v2" # FIXME1 (Add the model name from the above mentioned list)

In [ ]:
workdir = "workdir_object_detection" # FIXME2
host_url = "http://<ip_address>:<port_number>" # FIXME3 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME4 example: (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME5 set to True if you want to run automl for the model chosen in the previous cell

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

We will be using NVIDIA created `Synthetic Object detection data` based on KITTI dataset format in this notebook. To find more details about kitti format, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d).

**If using custom dataset; it should follow this dataset structure**
```
$DATA_DIR/train
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── labels
    ├── image_name_1.txt
    ├── image_name_2.txt
    ├── ...
$DATA_DIR/val
├── images
│   ├── image_name_5.jpg
│   ├── image_name_6.jpg
|   ├── ...
└── labels
    ├── image_name_5.txt
    ├── image_name_6.txt
    ├── ...
```
The file name should be same for images and labels folders

In [ ]:
dataset_to_be_used = "default" #FIXME6 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name # FIXME7
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
if dataset_to_be_used == "default":
    !python3 -m pip install awscli
    !aws s3 cp --no-sign-request s3://tao-object-detection-synthetic-dataset/tao_od_synthetic_train.tar.gz $DATA_DIR/
    !aws s3 cp --no-sign-request s3://tao-object-detection-synthetic-dataset/tao_od_synthetic_val.tar.gz $DATA_DIR/
    if model_name in ("efficientdet_tf1", "efficientdet_tf2", "deformable_detr", "dino"):
        print("Untarring file")
        !mkdir -p {DATA_DIR}/train
        !mkdir -p {DATA_DIR}/val
        !tar -xzf {DATA_DIR}/tao_od_synthetic_train.tar.gz -C {DATA_DIR}/train
        !tar -xzf {DATA_DIR}/tao_od_synthetic_val.tar.gz -C {DATA_DIR}/val

In [ ]:
if model_name in ("efficientdet_tf1", "efficientdet_tf2", "deformable_detr", "dino"):
    !python3 -m pip install ujson opencv-python tqdm
    import subprocess
    if not os.path.exists(os.path.join(DATA_DIR, "train")):
        raise Exception("Train dataset not present")
    if not os.path.exists(os.path.join(DATA_DIR, "val")):
        raise Exception("Eval dataset not present")
    
    #kitti to coco conversion for efficientdet
    if model_name == "efficientdet_tf2":
        label_map_extension = "yaml"
    else:
        label_map_extension = "txt"
    num_classes = subprocess.getoutput(f'python3 dataset_prepare/kitti/kitti_to_coco.py {DATA_DIR}/train/labels {DATA_DIR}/train {label_map_extension}')
    subprocess.getoutput(f'python3 dataset_prepare/kitti/kitti_to_coco.py {DATA_DIR}/val/labels {DATA_DIR}/val {label_map_extension}')
    !tar -C {DATA_DIR}/train -czf {DATA_DIR}/tao_od_synthetic_train.tar.gz images annotations.json label_map.{label_map_extension}
    !tar -C {DATA_DIR}/val -czf {DATA_DIR}/tao_od_synthetic_val.tar.gz images annotations.json label_map.{label_map_extension}
else:
    if dataset_to_be_used == "custom":
        !tar -C $DATA_DIR/train -czf $DATA_DIR/tao_od_synthetic_train.tar.gz images labels
        !tar -C $DATA_DIR/val -czf $DATA_DIR/tao_od_synthetic_val.tar.gz images labels

In [ ]:
ds_type = "object_detection"

if model_name in ("efficientdet_tf1", "efficientdet_tf2", "deformable_detr", "dino"):
    ds_format = "coco"
else:
    ds_format = "kitti"

train_dataset_path =  f"{DATA_DIR}/tao_od_synthetic_train.tar.gz"
eval_dataset_path = f"{DATA_DIR}/tao_od_synthetic_val.tar.gz"
test_dataset_path = f"{DATA_DIR}/tao_od_synthetic_val.tar.gz"

In [ ]:
# Create train dataset
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset with kitti"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset with kitti"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create testing dataset for inference
ds_format = "raw"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
test_dataset_id = response.json()["id"]

In [ ]:
# Upload
files = [("file",open(test_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{test_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Dataset convert Action <a class="anchor" id="head-3"></a>

In [ ]:
# Choose dataset convert action
if model_name not in ("deformable_detr", "dino"):
    if model_name in ("dssd", "ssd", "retinanet"):
        convert_action = "convert_and_index"
    elif "efficientdet" in model_name:
        convert_action = "convert_" + model_name
    else:
        convert_action = "convert"

In [ ]:
# Get default spec schema
if model_name not in ("deformable_detr", "dino"):
    endpoint = f"{base_url}/dataset/{dataset_id}/specs/{convert_action}/schema"

    response = requests.get(endpoint, headers=headers)

    print(response)
    # print(response.json()) ## Uncomment for verbose schema

    specs = response.json()["default"]

    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name not in ("deformable_detr", "dino"):
    if "efficientdet" in model_name:
        specs["dataset_convert"]["num_shards"] = 256
        specs["dataset_convert"]["tag"] = "train"
    else:
        specs["kitti_config"]["image_extension"] = ".jpg" #Change to png if your entire dataset is of png format

    if convert_action == "convert_and_index": # This can be applied for "convert" action also for networks like dnv2, yolo's, frcnn etc but not mandatory; for convert_and_index action it is mandatory
        # Map your classes to a superclass like mapping pedestrian to person or just the same name
        # Mention the classes in the dataset and their mapping
        specs["target_class_mapping"] = [{"key":"cone","value":"cone"},
                                 {"key":"cart","value":"cart"},
                                 {"key":"fire_extinguisher","value":"fire_extinguisher"},
                                 {"key":"forklift","value":"forklift"}]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Post spec
if model_name not in ("deformable_detr", "dino"):
    data = json.dumps(specs)

    endpoint = f"{base_url}/dataset/{dataset_id}/specs/{convert_action}"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
if model_name not in ("deformable_detr", "dino"):
    parent = None
    actions = [convert_action]
    data = json.dumps({"job":parent,"actions":actions})

    endpoint = f"{base_url}/dataset/{dataset_id}/job"

    response = requests.post(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

    train_ds_convert_id = response.json()[0]

In [ ]:
# Monitor job status by repeatedly running this cell
if model_name not in ("deformable_detr", "dino"):
    job_id = train_ds_convert_id
    endpoint = f"{base_url}/dataset/{dataset_id}/job/{job_id}"

    while True:
        clear_output(wait=True) 
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# Now, repeat the same for the eval dataset
# Get default spec schema
if model_name not in ("deformable_detr", "dino"):
    endpoint = f"{base_url}/dataset/{eval_dataset_id}/specs/{convert_action}/schema"

    response = requests.get(endpoint, headers=headers)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))


In [ ]:
# Apply changes
if model_name not in ("deformable_detr", "dino"):
    if "efficientdet" in model_name:
        specs["dataset_convert"]["num_shards"] = 256
        specs["dataset_convert"]["tag"] = "val"
    else:
        specs["kitti_config"]["image_extension"] = ".jpg" #Change to png if your entire dataset is of png format

    if convert_action == "convert_and_index": # This can be applied for "convert" action also for networks like dnv2, yolo's, frcnn etc but not mandatory; for convert_and_index action it is mandatory
        #Map your classes to a superclass like mapping pedestrian to person or just the same name
        #Mention the classes in the dataset and their mapping
        specs["target_class_mapping"] = [{"key":"cone","value":"cone"},
                                 {"key":"cart","value":"cart"},
                                 {"key":"fire_extingusher","value":"fire_extingusher"},
                                 {"key":"forklift","value":"forklift"}]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Post spec
if model_name not in ("deformable_detr", "dino"):
    data = json.dumps(specs)

    endpoint = f"{base_url}/dataset/{eval_dataset_id}/specs/{convert_action}"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
if model_name not in ("deformable_detr", "dino"):
    parent = None
    actions = [convert_action]
    data = json.dumps({"job":parent,"actions":actions})

    endpoint = f"{base_url}/dataset/{eval_dataset_id}/job"

    response = requests.post(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

    eval_ds_convert_id = response.json()[0]

In [ ]:
# Monitor job status by repeatedly running this cell
if model_name not in ("deformable_detr", "dino"):
    job_id = eval_ds_convert_id
    endpoint = f"{base_url}/dataset/{eval_dataset_id}/job/{job_id}"

    while True:
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
encode_key = "tlt_encode"
data = json.dumps({"network_arch":model_name,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":test_dataset_id,
                       "calibration_dataset":dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for PTM on NGC for the Object Detection model chosen

In [ ]:
# List all pretrained models for the chosen network architecture
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

for rsp in response_json:
    if rsp["network_arch"] == model_name:
        if "encryption_key" not in rsp.keys():
            print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
# Assigning pretrained models to different object detection models versions
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"detectnet_v2" : "detectnet_v2:resnet18",
                  "deformable_detr": "pretrained_deformable_detr_nvimagenet:resnet50",
                  "dino": "pretrained_dino_nvimagenet:resnet50",
                  "dssd" : "pretrained_object_detection:resnet18",
                  "efficientdet_tf1" : "pretrained_efficientdet:efficientnet_b0",
                  "efficientdet_tf2" : "pretrained_efficientdet_tf2:efficientnet_b0",
                  "faster_rcnn" : "pretrained_object_detection:resnet18",
                  "retinanet" : "pretrained_object_detection:resnet18",
                  "ssd" : "pretrained_object_detection:resnet18",
                  "yolo_v3" : "pretrained_object_detection:resnet18",
                  "yolo_v4" : "pretrained_object_detection:resnet18",
                  "yolo_v4_tiny": "pretrained_object_detection:cspdarknet_tiny"}
no_ptm_models = set([])

In [ ]:
# Get pretrained model
if model_name not in no_ptm_models:
    model_list = f"{base_url}/model"
    response = requests.get(model_list, headers=headers)

    response_json = response.json()

    # Search for ptm with given ngc path
    ptm = []
    for rsp in response_json:
        if rsp["network_arch"] == model_name and rsp["ngc_path"].endswith(pretrained_map[model_name]):
            ptm_id = rsp["id"]
            ptm = [ptm_id]
            print("Metadata for model with requested NGC Path")
            print(rsp)
            break

In [ ]:
if model_name not in no_ptm_models:
    ptm_information = {"ptm":ptm}
    data = json.dumps(ptm_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-8"></a>

In [ ]:
if automl_enabled:
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/train/schema"
    response = requests.get(endpoint, headers=headers)
    specs = response.json()["automl_default_parameters"]
    print(json.dumps(specs, sort_keys=True, indent=4))

### Set AutoML related configurations <a class="anchor" id="head-9"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters:

[DetectNet_V2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/detectnet_v2/detectnet_v2%20-%20train.csv), 
[Deformable Detr](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/deformable_detr/deformable_detr%20-%20train.csv), 
[DINO](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/dino/dino%20-%20train.csv), 
[DSSD](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/dssd/dsssd%20-%20train.csv), 
[EfficientDet TF1](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/efficientdet_tf1/efficientdet_tf1%20-%20train.csv), 
[EfficientDet TF2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/efficientdet_tf2/efficientdet_tf2%20-%20train.csv), 
[FasterRCNN](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/faster_rcnn/faster_rcnn%20-%20train.csv), 
[RetinaNet](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/retinanet/retinanet%20-%20train.csv), 
[SSD](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/ssd/ssd%20-%20train.csv), 
[YOLO_V3](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/yolo_v3/yolo_v3%20-%20train.csv), 
[YOLO_V4](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/yolo_v4/yolo_v4%20-%20train.csv), 
[YOLO_V4_Tiny](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/yolo_v4_tiny/yolo_v4_tiny%20-%20train.csv)

In [ ]:
if automl_enabled:
    # Choose automl algorithm between "Bayesian" and "HyperBand".
    automl_algorithm="Bayesian" # FIXME8 example: Bayesian/HyperBand

    # for Bayesian; don't change, more metrics will be supported in the future; for Hyperband, only loss is monitored, so this doesn't matter
    metric = "kpi"

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    automl_information = {"automl_enabled":automl_enabled,
                          "automl_algorithm":automl_algorithm,
                          "epoch_multiplier": 1, # Will be considered for Hyperband only
                          "metric":metric,
                          "automl_add_hyperparameters":str(additional_automl_parameters),
                          "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                         }
    data = json.dumps(automl_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-10"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### Train <a class="anchor" id="head-11"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes for any of the parameters listed in the previous cell as required
if model_name in ("deformable_detr", "dino"):
    specs["dataset"]["num_classes"] = int(num_classes) + 1
    specs["train"]["num_epochs"] = 10
    specs["train"]["num_gpus"] = 1
    
elif model_name == "efficientdet_tf1":
    specs["training_config"]["num_epochs"] = 10
    specs["training_config"]["train_batch_size"] = 2
    specs["training_config"]["num_examples_per_epoch"] = 1414 #number of images in your dataset/number of gpu's
    specs["dataset_config"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion
    specs["eval_config"]["eval_epoch_cycle"] = 10
    specs["gpus"] = 1

elif model_name == "efficientdet_tf2":
    specs["train"]["num_epochs"] = 10
    specs["gpus"] = 1
    specs["train"]["batch_size"] = 4
    specs["train"]["num_examples_per_epoch"] = 1414 #number of images in your dataset/number of gpu's
    specs["dataset"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion

else:
    # Example for detectnet_v2 (for each network the parameter key might be different)
    specs["training_config"]["num_epochs"] = 10 # num_epochs is the parameter name for all object detection networks
    specs["gpus"] = 1

if "dataset_config" in specs.keys() and "image_extension" in specs["dataset_config"].keys():
    specs["dataset_config"]["image_extension"] = "jpg"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
# For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status by repeatedly running this cell' cell above (4th cell above from this cell)
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
# Download job contents once the above job shows "Done" status
# Download output of train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments

if automl_enabled:
    !python3 -m pip install pandas==1.5.1
    import pandas as pd
    model_downloaded_path = f"{model_downloaded_path}/best_model"

if os.path.exists(model_downloaded_path):        
    #List the binary model file
    print("\nCheckpoints for the training experiment")
    if os.path.exists(model_downloaded_path+"/train/weights") and len(os.listdir(model_downloaded_path+"/train/weights")) > 0:
        print(f"Folder: {model_downloaded_path}/train/weights")
        print("Files:", os.listdir(model_downloaded_path+"/train/weights"))
    elif os.path.exists(model_downloaded_path+"/weights") and len(os.listdir(model_downloaded_path+"/weights")) > 0:
        print(f"Folder: {model_downloaded_path}/weights")
        print("Files:", os.listdir(model_downloaded_path+"/weights"))
    else:
        print(f"Folder: {model_downloaded_path}")
        print("Files:", os.listdir(model_downloaded_path))

    if automl_enabled:
        experiment_artifacts = json.load(open(f"{model_downloaded_path}/controller.json","r"))
        data_frame = pd.DataFrame(experiment_artifacts)
        # Print experiment id/number and the corresponding result
        print("\nResults of all experiments")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
            print(data_frame[["id","result"]])

### Evaluate <a class="anchor" id="head-12"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name == "efficientdet_tf1":
    specs["dataset_config"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion
elif model_name == "efficientdet_tf2":
    specs["evaluate"]["num_samples"] = 353 #number of images in your dataset
    specs["dataset"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion
elif model_name in ("deformable_detr", "dino"):
    specs["dataset"]["num_classes"] = int(num_classes) + 1

if "dataset_config" in specs.keys() and "image_extension" in specs["dataset_config"].keys():
    specs["dataset_config"]["image_extension"] = "jpg"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["evaluate"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['evaluate']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Optimize <a class="anchor" id="head-13"></a>

- We optimize the trained model by pruning and retraining in the following cells

### Apply specs for prune <a class="anchor" id="head-14"></a>

In [ ]:
# Get default spec schema
if model_name not in ("deformable_detr", "dino"):
    endpoint = f"{base_url}/model/{model_id}/specs/prune/schema"

    response = requests.get(endpoint, headers=headers)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
# None for prune

In [ ]:
# Post spec
if model_name not in ("deformable_detr", "dino"):
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/prune"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Apply specs for retrain <a class="anchor" id="head-15"></a>

In [ ]:
# Get default spec schema
if model_name not in ("deformable_detr", "dino"):
    endpoint = f"{base_url}/model/{model_id}/specs/retrain/schema"

    response = requests.get(endpoint,headers=headers)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes for any of the parameters listed in the previous cell as required
if model_name not in ("deformable_detr", "dino"):
    # for efficientdet_tf1
    if model_name == "efficientdet_tf1":
        specs["training_config"]["num_epochs"] = 10
        specs["gpus"] = 1
        specs["training_config"]["train_batch_size"] = 8
        specs["training_config"]["num_examples_per_epoch"] = 1414 #number of images in your dataset/number of gpu's
        specs["dataset_config"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion
        specs["eval_config"]["eval_epoch_cycle"] = 10

    # for efficientdet_tf2
    elif model_name == "efficientdet_tf2":
        specs["train"]["num_epochs"] = 10
        specs["gpus"] = 1
        specs["train"]["batch_size"] = 4
        specs["train"]["num_examples_per_epoch"] = 1414 #number of images in your dataset/number of gpu's
        specs["dataset"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion

    # Example for (for each network the parameter key might be different)
    else:
        specs["training_config"]["num_epochs"] = 10 # num_epochs is the parameter name for all object detection networks
        specs["gpus"] = 1

    if "dataset_config" in specs.keys() and "image_extension" in specs["dataset_config"].keys():
        specs["dataset_config"]["image_extension"] = "jpg"

In [ ]:
# Post spec
if model_name not in ("deformable_detr", "dino"):
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/retrain"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Run Actions <a class="anchor" id="head-16"></a>

We use the API's job chaining feature to prune, retrain and evaluate the retrained model

In [ ]:
# Run actions
if model_name not in ("deformable_detr", "dino"):
    parent = job_map["train"]
    actions = ["prune","retrain","evaluate"]
    data = json.dumps({"job":parent,"actions":actions})

    endpoint = f"{base_url}/model/{model_id}/job"

    response = requests.post(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

    job_map["prune"] = response.json()[0]
    job_map["retrain"] = response.json()[1]
    job_map["eval_retrain"] = response.json()[2]
    print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell (prune)
if model_name not in ("deformable_detr", "dino"):
    job_id = job_map['prune']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (retrain)
if model_name not in ("deformable_detr", "dino"):
    job_id = job_map['retrain']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (evaluate)
if model_name not in ("deformable_detr", "dino"):
    job_id = job_map['eval_retrain']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# Optional cancel job - for jobs that are pending/running (retrain)

# if model_name not in ("deformable_detr", "dino"):
#     job_id = job_map['retrain']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
# Optional delete job - for jobs that are error/done (retrain)

# if model_name not in ("deformable_detr", "dino"):
#     job_id = job_map['retrain']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

#     response = requests.delete(endpoint, headers=headers)

#     print(response)
#     print(response.json())

### Export <a class="anchor" id="head-17"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name == "efficientdet_tf2":
    specs["dataset"]["num_classes"] = int(num_classes)
elif model_name in ("deformable_detr", "dino"):
    specs["dataset"]["num_classes"] = int(num_classes) + 1

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["export"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    
print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for the generated .onnx file
!ls {model_downloaded_path}

### TRT Engine generation using TAO-Deploy <a class="anchor" id="head-19"></a>

- Here, we use the exported model to generate trt engine

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name == "efficientdet_tf2":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "int8"
    specs["dataset"]["num_classes"] = int(num_classes)
elif model_name in ("deformable_detr", "dino"):
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "int8"
    specs["dataset"]["num_classes"] = int(num_classes) + 1
else:
    specs["data_type"] = "int8"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["export"]
actions = ["gen_trt_engine"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["model_gen_trt_engine"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['model_gen_trt_engine']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-20"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs dictionary if necessary
if model_name == "efficientdet_tf1":
    specs["dataset_config"]["num_classes"] = int(num_classes) #num_classes was computed during kitti_to_coco_conversion
elif model_name == "efficientdet_tf2":
    specs["dataset"]["num_classes"] = int(num_classes)
elif model_name in ("deformable_detr", "dino"):
    specs["dataset"]["num_classes"] = int(num_classes) + 1

if "dataset_config" in specs.keys() and "image_extension" in specs["dataset_config"].keys():
    specs["dataset_config"]["image_extension"] = "jpg"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_tlt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_tlt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
from IPython.display import Image
import glob
sample_image = glob.glob(f"{inference_out_path}/**/*.jpg", recursive=True)[6]
Image(filename=sample_image) 

### TRT inference <a class="anchor" id="head-21"></a>

- no need to change the specs since we already uploaded it at the tlt inference step

In [ ]:
# Run action
parent = job_map["model_gen_trt_engine"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_trt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_trt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_trt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
from IPython.display import Image
import glob
sample_image = glob.glob(f"{inference_out_path}/**/*.jpg", recursive=True)[6]
Image(filename=sample_image) 